In [23]:
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time, threading, os, requests,re

In [ ]:
def scarpe(path):
    options = ChromeOptions()
    options.headless=True
    service = Service(ChromeDriverManager().install())
    driver = Chrome(service=service, options=options)
    driver.get(path)
    driver.maximize_window()
    

    try:
        # log_card =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="site-nav__account"]'))) 
        login = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/a[text() = "Login"]')))
        if login:
            login.click()
    except Exception as e:
        print("no login")


    try:
        provided_u_name = "fourbrotherstrading@icloud.com"
        user_name = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID,'username')))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")
    

    try:
        provided_pass = "Sultanmirza1501#"
        password = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.ID, 'password')))
        driver.execute_script("arguments[0].scrollIntoView();", password)
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}") 
    

    try:
        check = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/button[text() = "Sign in"]')))
        if check:
           
            check.click()
    except Exception as e:
        print(f"No check tab found and the error is {e}")
        
    
    try:
        auction_name = WebDriverWait(driver, 3).until(
            EC.presence_of_element_located((By.XPATH, '//div[@class="col-md-8"]/h1'))
        ).text.strip()
        auction_name = " ".join(auction_name.split())

    except Exception as e:
        print("Auction name not found:", e)
    
    

    try:

        cars = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-4 motorvehicle-pagination-block md-align-right mt"]/p)[1]'))).text.strip("()")
        if cars:
 
            splited_cars = cars.split(" ")
            total_cars = int(splited_cars[-2])
            print(f"{total_cars} number of cars found")
        else:
            print("No cars found")
    except Exception as e:
        print(f"No cars found and error is {e}")
    
    
    # result
    results = []
    car_count = 0
    visited_pages = set()
    while car_count < total_cars: #total_cars
        
        try:
            page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="row mb--"]')))
            
            for i in range(len(page_cars)): # len(page_cars)
                if car_count >= total_cars:
                    break
                
                try:
                    page_cars = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="row mb--"]')))
                    driver.execute_script("arguments[0].scrollIntoView();", page_cars[i])
                    time.sleep(1)
                    page_cars[i].click()

                    details = {}  
                    
                    # title and lot --> done
                    details['Auction Name'] = auction_name if auction_name else ''
                    try:
                        tit_lot = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="vehicle-title"]'))).text.strip()
                        if tit_lot:
                            tit_lot_splited = tit_lot.split(":")
                            details['Title'] = tit_lot_splited[1]
                            details['Lot'] = tit_lot_splited[0]
                        else:
                            details['Title'] = 'na'
                            details['Lot'] = 'na'
                            # print("No car title and lot number")
                    except Exception as e:
                        print("No car title and lot number")
                    
                    # location, date and time
                    try:
                        location = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//span[@style="white-space: nowrap;"])[2]'))).text.strip()
                        if location:
                            details['Center'] = location
                        else:
                            details['Center'] = 'na'
                        
                        dt = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//span[@style="white-space: nowrap;"])[3]'))).text.strip()
                        if dt:
                            details['Start Date'] = dt
                        else:
                            details['Start Date'] = 'na'
                            
                        t = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, '(.//span[@style="white-space: nowrap;"])[4]'))).text.strip()
                        if t:
                            details['Start Time'] = t
                        else:
                            details['Start Time'] = 'na'
                    except Exception as e:
                        print("No date time and location details")
                        
                    # images --> done
                    try:
                        img_card =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@class="slick-list draggable"]'))) 
                        if img_card:
                            imgs =WebDriverWait(img_card, 2).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img'))) 
                            if imgs:
                                imgs_lst = [img.get_attribute("src") for img in imgs]
                                details['Images'] = ", ".join(imgs_lst)
                            else:
                                print("No images")
                    except Exception as e:
                        print(f"No image card {e}")
                    
                    # inspection
                    try:
                        inspec =WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, './/a[text()= "View PDF"]'))).get_attribute("href")
                        if inspec:
                            details['Inspection Report'] = inspec
                        else:
                            details['Inspection Report'] = 'na'
                    except Exception as e:
                        print("No inspection report")
                        
                    grade = "na"

                    try:
                        grade_img = WebDriverWait(driver, 5).until(
                            EC.presence_of_element_located((By.XPATH, '//img[contains(@src, "namagrades")]'))
                        )

                        src = grade_img.get_attribute("src")  # full URL milega
                        match = re.search(r'(\d+)\.jpg', src)

                        if match:
                            grade = match.group(1)

                    except Exception as e:
                        print("Grade not found:", e)

                    print("Grade:", grade)

                    details["Grade"] = grade
                        
                    # car basic dets
                    try:
                        dets_main = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="container"])[3]')))
                        if dets_main:
                            upper_det = WebDriverWait(dets_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="row mb+"])[1]')))
                            try:
                                det1_body = upper_det.find_elements(By.XPATH, './/table[@class="table table-condensed vehicle-table"]')
                                if det1_body:
                                    for det1_b in det1_body:
                                        det1_trs = WebDriverWait(det1_b, 2).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                        for det1_tr in det1_trs:
                                            det1_lbls = det1_tr.find_elements(By.TAG_NAME, 'th')
                                            det1_lbl = det1_lbls[0].text.strip() if det1_lbls else None
                                            
                                            det1_vals = det1_tr.find_elements(By.TAG_NAME, 'td')
                                            det1_val = det1_vals[0].text.strip() if det1_vals else None
                                            if det1_lbl or det1_val:
                                                details[det1_lbl] = det1_val 
                            except Exception as e:
                                print("No det1")                   
                            
                            # danger notes
                            try:
                                danger = WebDriverWait(dets_main, 2).until(EC.presence_of_element_located((By.XPATH, './/div[@class="panel panel-danger"]')))
                                if danger:
                                    dang_head = danger.find_element(By.XPATH, './/div[@class="panel-heading"]').text.strip()
                                    dang_val = danger.find_element(By.TAG_NAME, 'strong').text.strip()
                                    if dang_head and dang_val:
                                        details[dang_head] = dang_val
                                else:
                                    print("No danger tag found")
                            except Exception as e:
                                print("No danger data")
                            
                            # economy
                            economy = WebDriverWait(dets_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="row mb+"])[2]')))
                            try:
                                eco_body = economy.find_elements(By.XPATH, './/table[@class="table table-condensed vehicle-table"]')
                                if eco_body:
                                    for eco_b in eco_body:
                                        eco_trs = WebDriverWait(eco_b, 2).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                        for eco_tr in eco_trs:
                                            eco_lbls = eco_tr.find_elements(By.TAG_NAME, 'th')
                                            eco_lbl = eco_lbls[0].text.strip() if eco_lbls else None
                                            
                                            eco_vals = eco_tr.find_elements(By.TAG_NAME, 'td')
                                            eco_val = eco_vals[0].text.strip() if eco_vals else None
                                            if eco_lbl or eco_val:
                                                details[eco_lbl] = eco_val 
                            except Exception as e:
                                print("No economy")
                                

                            # service hist
                            service = WebDriverWait(dets_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="row mb+"])[3]')))
                            try:
                                serv_body = service.find_elements(By.XPATH, './/div[@class="col-md-4"]')
                                if serv_body:
                                    for serv_b in serv_body:
                                        serv_lbl = serv_b.find_element(By.TAG_NAME, 'label').text.strip()
                                        serv_val = serv_b.find_element(By.TAG_NAME, 'p').text.strip()
                                        if serv_lbl or serv_val:
                                            details[serv_lbl] = serv_val 
                            except Exception as e:
                                print("No service")
                                
                            # features --> done
                            try:
                                fets = dets_main.find_element(By.XPATH, ".//h2[text()='Features']/following-sibling::div[contains(@class, 'row mb+')][1]")
                                if fets:
                                    fet_lst = []
                                    fet_trs = WebDriverWait(fets, 2).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                                    for fet_tr in fet_trs:
                                        fet_vals = fet_tr.find_element(By.TAG_NAME, 'td').text.strip()
                                        fet_lst.append(fet_vals)
                                    details['Features'] = fet_lst
                            except Exception as e:
                                print("No fets")

                            # cond images
                            try:
                                cond_card = WebDriverWait(dets_main, 3).until(EC.presence_of_element_located((By.XPATH, ".//h2[text()='Condition Check']/following-sibling::div[contains(@class, 'images gallerywidget-condition gallerywidget mb')]")))
                                driver.execute_script("arguments[0].scrollIntoView();", cond_card)
                                time.sleep(3)
                                if cond_card:
                                    cond_imgs = cond_card.find_elements(By.TAG_NAME, 'img')
                                    if cond_imgs:
                                        cond_img_lst = [cond_img.get_attribute("src") for cond_img in cond_imgs]
                                        details['Damaged_images'] = ", ".join(cond_img_lst)
                            except Exception as e:
                                print("No cond images")                            
                            
                            try:
                                damage_parent = WebDriverWait(dets_main, 3).until(
                                    EC.presence_of_element_located(
                                        (By.XPATH, ".//div[@class='gallerywidget-descriptions']")
                                    )
                                )

                                damage_items = damage_parent.find_elements(By.CLASS_NAME, "gallerywidget-description")

                                damage_list = []

                                for item in damage_items:

                                    # Number
                                    try:
                                        number = item.find_element(By.CLASS_NAME, "description-number").text.strip()
                                    except:
                                        number = "NA"

                                    # Title inside <strong>
                                    try:
                                        title1 = item.find_element(By.TAG_NAME, "strong").text.strip()
                                    except:
                                        title1 = "NA"

                                    # Description lines
                                    try:
                                        desc_block = item.find_element(By.CLASS_NAME, "description-item-content").text.strip()
                                        desc_lines = [x.strip() for x in desc_block.split("\n")]

                                        issue1 = desc_lines[1] if len(desc_lines) > 1 else "NA"
                                        issue2 = desc_lines[2] if len(desc_lines) > 2 else "NA"
                                    except:
                                        issue1, issue2 = "NA", "NA"

                                    # Optional image URL
                                    img_url = item.get_attribute("data-url") or ""

                                    # Final combined entry
                                    damage_list.append(f"{number} - {title1} - {issue1} - {issue2}")

                                if damage_list:
                                    details["Damage Details"] = ", ".join(damage_list)

                            except Exception as e:
                                print("No damage detail section")

                            
                            
                            # interior
                            try:
                                interior_main = WebDriverWait(dets_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="row mb+"])[2]')))
                                if interior_main:
                                    interior = WebDriverWait(interior_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[1]')))

                                    table_body = interior.find_element(By.XPATH, '(.//table[@class="table table-condensed vehicle-table"]/tbody)')              
                                    # Get all rows in the table
                                    rows = table_body.find_elements(By.TAG_NAME, 'tr')
                                    for row in rows:
                                        # Get the th and td elements
                                        key = row.find_element(By.TAG_NAME, 'th').text.strip()
                                        value = row.find_element(By.TAG_NAME, 'td').text.strip()
                                        if key and value:
                                            details[key] = value
                            except Exception as e:
                                print(f"No data interior")
                            
                            # tyres
                            # try:
                            #     tyre_main = WebDriverWait(dets_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[2]/table/tbody')))
                            #     if tyre_main:
                            #         tyre_trs = WebDriverWait(tyre_main, 2).until(EC.presence_of_all_elements_located((By.TAG_NAME, 'tr')))
                            #         if tyre_trs:
                            #             for tyre_tr in tyre_trs:
                            #                 tyre_lbl = tyre_tr.find_elements(By.TAG_NAME, "td")
                            #                 if tyre_lbl:
                            #                     details[tyre_lbl[0].text.strip()] = [tyre_lbl[1].text.strip(), tyre_lbl[2].text.strip()]

                            #         else:
                            #             print("No interior")
                            # except Exception as e:
                            #     print(f"No tyre data")
                            
                            try:
                                tyre_main = WebDriverWait(dets_main, 2).until(
                                    EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-6"])[2]/table/tbody'))
                                )

                                tyre_conditions = []  

                                if tyre_main:
                                    tyre_trs = WebDriverWait(tyre_main, 2).until(
                                        EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
                                    )

                                    for tyre_tr in tyre_trs:
                                        tds = tyre_tr.find_elements(By.TAG_NAME, "td")

                                        if len(tds) >= 3:
                                            key = tds[0].text.strip()    
                                            val1 = tds[1].text.strip() 
                                            val2 = tds[2].text.strip() 

                                            tyre_conditions.append(f"{key}: {val1}, {val2}")

               
                                if tyre_conditions:
                                    details["Tyres Condition"] = " | ".join(tyre_conditions)  
                                    # OR use comma:
                                    # details["Tyres Condition"] = ", ".join(tyre_conditions)

                            except Exception as e:
                                print("No tyre data")

                            service_history_list = []

                            try:
                                # Service History ka row div dhundho
                                service_items = WebDriverWait(driver, 3).until(
                                    EC.presence_of_all_elements_located(
                                        (By.XPATH, ".//h2[text()='Service History']/following-sibling::div//div[contains(@class,'col-md-4')]")
                                    )
                                )

                                for item in service_items:
                                    label = item.find_element(By.TAG_NAME, "label").text.strip()
                                    value = item.find_element(By.TAG_NAME, "p").text.strip()
                                    service_history_list.append(f"{label}: {value}")

                            except Exception as e:
                                print("No Service History found:", e)

                            # Comma-separated string me convert
                            service_history_str = ", ".join(service_history_list)

                            # Variable me store
                            details['Service History'] = service_history_str

                            print(details['Service History'])

                            # engine
                            try:
                                engine_main = WebDriverWait(dets_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="row mb+"])[6]')))
                                if engine_main:
                                    engine = WebDriverWait(engine_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-4"])[1]')))

                                    engine_body = engine.find_element(By.XPATH, '(.//table[@class="table table-condensed vehicle-table"]/tbody)')              
                                    # Get all rows in the table
                                    engine_rows = engine_body.find_elements(By.TAG_NAME, 'tr')
                                    for engine_row in engine_rows:
                                        # Get the th and td elements
                                        engine_key = engine_row.find_element(By.TAG_NAME, 'th').text.strip()
                                        engine_value = engine_row.find_element(By.TAG_NAME, 'td').text.strip()
                                        if engine_key and engine_value:
                                            details[engine_key] = engine_value
                            except Exception as e:
                                print(f"No data for engine")
                            
                            # Dashboard
                            try:
                                dash_main = WebDriverWait(dets_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="row mb+"])[6]')))
                                if dash_main:
                                    dash = WebDriverWait(dash_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-4"])[2]')))
                                    dash_body = dash.find_element(By.XPATH, '(.//table[@class="table table-condensed vehicle-table"]/tbody)')              
                                    # Get all rows in the table
                                    dash_rows = dash_body.find_elements(By.TAG_NAME, 'tr')
                                    for dash_row in dash_rows:
                                        # Get the th and td elements
                                        dash_key = dash_row.find_element(By.TAG_NAME, 'th').text.strip()
                                        dash_value = dash_row.find_element(By.TAG_NAME, 'td').text.strip()
                                        if dash_key and dash_value:
                                            details[dash_key] = dash_value

                            except Exception as e:
                                print(f"No data for dashboard")
                            
                            # Other
                            try:
                                other_main = WebDriverWait(dets_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="row mb+"])[6]')))
                                if other_main:
                                    other = WebDriverWait(other_main, 2).until(EC.presence_of_element_located((By.XPATH, '(.//div[@class="col-md-4"])[3]')))
                                    other_body = other.find_element(By.XPATH, '(.//table[@class="table table-condensed vehicle-table"]/tbody)')              
                                    # Get all rows in the table
                                    other_rows = other_body.find_elements(By.TAG_NAME, 'tr')
                                    for other_row in other_rows:
                                        # Get the th and td elements
                                        other_key = other_row.find_element(By.TAG_NAME, 'th').text.strip()
                                        other_value = other_row.find_element(By.TAG_NAME, 'td').text.strip()
                                        if other_key and other_value:
                                            details[other_key] = other_value
                            except Exception as e:
                                print(f"No data for other")
                                
                        else:
                            print("no dets main found")      
                       
                    except Exception as e:
                        print(f"No car dets found")
                    
                    car_count+=1
                    results.append(details)
                    driver.back()
                    
                except Exception as e:
                    print("No car to clk")
                
            if car_count % 30 == 0: # --> 25%25 == 0 and 50%25== 0 --> issue here
                try:
                    next_card =WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, './/div[@class="col-md-6 motorvehicle-pagination-block md-align-right"]'))) 
                    next_link = WebDriverWait(next_card, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/a[contains(@class, "btn") and not(contains(@class, "disabled"))]')))
                    if next_link:
                        for nxt_btn in next_link:
                            page_url = nxt_btn.get_attribute("href")
                            if page_url and page_url not in visited_pages:  # Check if the page has not been visited
                                # print(f"Visiting: {page_url}")
                                visited_pages.add(page_url)  # Mark it as visited
                                nxt_btn.click()
                        
                                WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.XPATH, './/div[@class="row mb--"]')))
                                print("Hit the next button")
                                break
                except Exception as e:
                    print("No next found")
                    break
        
        except Exception as e:
            print("No more cars")
            break
    
    df = pd.DataFrame.from_dict(results)
    df.to_csv("data.csv", index=False)
    # time.sleep(2)
    driver.quit()
path = "https://www.cityauctiongroup.com/auction/1994"
scarpe(path)

286 number of cars found
Grade: 2
No data interior
Last Service Date:: 03/07/2025, Number of Stamps:: 4, Last Service Miles:: 118,378
Grade: 2
No data interior
Last Service Date:: 03/07/2025, Number of Stamps:: 4, Last Service Miles:: 118,378
No car title and lot number
No date time and location details
No image card Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=141.0.7390.123)
Stacktrace:
	GetHandleVerifier [0x0xbcfe43+66515]
	GetHandleVerifier [0x0xbcfe84+66580]
	(No symbol) [0x0x9bdc48]
	(No symbol) [0x0x99c18d]
	(No symbol) [0x0xa31a4e]
	(No symbol) [0x0xa4c4d9]
	(No symbol) [0x0xa2afc6]
	(No symbol) [0x0x9fc2ca]
	(No symbol) [0x0x9fd154]
	GetHandleVerifier [0x0xe27353+2521315]
	GetHandleVerifier [0x0xe222d3+2500707]
	GetHandleVerifier [0x0xbf7c94+229924]
	GetHandleVerifier [0x0xbe81f8+165768]
	GetHandleVerifier [0x0xbeecad+193085]
	GetHandleVerifier [0x0xbd8158+100072]
	GetHandleVerifier [0x0xbd82f0+100480]
	Ge

In [25]:
df = pd.read_csv("data.csv")
df.rename(columns={"Registration":"Reg", "Registered":"D.O.R", "Manufacturer":"Make", "Previous Owners":"Former Keepers", "Number of Stamps:":"No of services", "Last Service Miles:":"Last service mileage", "Fuel":"Fuel Type", "Last Service Date:":"Last Service", "VAT":"VAT Status", "CO2":"CO2 Emissions", "NOx":"NOx Emissions", "Roof / sunroof":"Sunroof", "Date/Time":"Inspection Date"}, inplace=True)
df['Mileage warranted']  = df['Mileage'].apply(lambda x : x.split(" ")[-1] if isinstance(x, str) and " " in x else None)
df['Lot'] = df['Lot'].apply(lambda x : x.split(" ")[-1] if isinstance(x, str) and " " in x else None)
df['Euro Status']  = df['Standard Euro Emissions'].apply(lambda x : x.split(" ")[-1] if isinstance(x, str) and " " in x else None)
df.drop(columns=['Standard Euro Emissions'], inplace=True)

cols_to_extract = ['Condition', 'Notes', 'Carpet condition', 'Seat condition', 'Upholstery condition']
existing_cols = [col for col in cols_to_extract if col in df.columns]
# Build the 'Additional Information' column
df['Additional Information'] = df[existing_cols].apply(lambda row: {col: row[col] if pd.notnull(row[col]) else 'na' for col in existing_cols},axis=1)

df['CC'] = df['CC'].apply(lambda x: f"{int(x.replace(",", "")) / 1000:.1f}" if isinstance(x, str) else f"{x / 1000:.1f}")

df.drop(columns=existing_cols, inplace=True)
df.to_csv("cag_data.csv", index=False)
# os.remove("data.csv")

In [26]:
import pandas as pd

df = pd.read_csv("cag_data.csv")
rename_map = {
    "Engine Runs": "Non Runner",
}
df.rename(columns=rename_map, inplace=True)
df.to_csv("cag_data.csv", index=False)


### Downloading

In [27]:
from PIL import Image, ImageDraw, ImageFont
def add_watermark_to_image(image_path, text="Sourced from City Auction Group"):
    try:
        image = Image.open(image_path).convert("RGBA")
        txt_layer = Image.new("RGBA", image.size, (255, 255, 255, 0))
        draw = ImageDraw.Draw(txt_layer)

        # Load font
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except:
            font = ImageFont.load_default()

        # Calculate text size and position
        margin = 10
        bbox = draw.textbbox((0, 0), text, font=font)
        text_width = bbox[2] - bbox[0]
        text_height = bbox[3] - bbox[1]
        x = image.width - text_width - margin
        y = image.height - text_height - margin

        # Draw semi-transparent background box
        box_width = text_width + 2 * margin
        box_height = text_height + 2 * margin
        draw.rectangle([x - margin, y - margin, x - margin + box_width, y - margin + box_height],fill=(0, 0, 0, 160))

        # Draw watermark text
        draw.text((x, y), text, font=font, fill=(255, 255, 255, 200))

        # Merge and save
        watermarked = Image.alpha_composite(image, txt_layer).convert("RGB")
        watermarked.save(image_path)
        print(f"Watermark added to {image_path}")

    except Exception as e:
        print(f"Failed to watermark {image_path}: {e}")


In [28]:
from urllib.parse import urlparse, urljoin
# urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# urljoin: Joins a base URL with a relative path to form a complete URL.

df = pd.read_csv("cag_data.csv")
reg_img = df[["Reg", "Images"]]
cond_img = df[["Reg", "Damaged_images"]]
report = df[["Reg",'Inspection Report']]

def download_images(data, main_folder="Images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg"] # separate reg nums
        
        if pd.isna(row["Images"]) or not str(row["Images"]).strip():
            print(f"Skipping {reg_no} due to missing images.")
            # continue  # Skip this row
        else:
        
            image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
            
            # Create a subfolder for the car registration number
            reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
            os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
            
            # loop around the urls of the current row and also save index for further use
            for idx, url in enumerate(image_urls):
                url = url.strip()  # Remove extra spaces
                if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                    url = urljoin("https://", url) # sets the urls starting from https://.....
                
                # parse the url
                parsed_url = urlparse(url)

                # check if the parsed url is valid
                if not parsed_url.scheme or not parsed_url.netloc:
                    print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                    # continue
                else:
                
                    # try downloading images
                    try:
                        # Download the image
                        response = requests.get(url, stream=True) # send the url for downloading
                        response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                        
                        # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                        # If no file name is found, assigns a default name based on the index.
                        file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                        # Extracts the file extension (e.g., jpg). from the last index value
                        file_extension = file_name.split(".")[-1]
                        
                        # Ensure the file has a valid extension
                        if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                            file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                        
                        # Construct the full path for the file
                        full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}.jpg")
                        
                        # Save the image
                        with open(full_file_name, 'wb') as f:
                            for chunk in response.iter_content(1024):
                                f.write(chunk)

                        # Add watermark
                        add_watermark_to_image(full_file_name)
                        
                        print(f"Downloaded: {full_file_name}")
                    except Exception as e:
                        print(f"Failed to download {url} for {reg_no}: {e}")
# cond images
def download_cond(data, main_folder="Damaged_images"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg"] # separate reg nums
        
        if pd.isna(row["Damaged_images"]) or not str(row["Damaged_images"]).strip():
            print(f"Skipping {reg_no} due to missing images.")
            # continue  # Skip this row
        else:
        
            image_urls = row["Damaged_images"].split(", ")  # Split image URLs by comma if multiple
            
            # Create a subfolder for the car registration number
            reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
            os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
            
            # loop around the urls of the current row and also save index for further use
            for idx, url in enumerate(image_urls):
                url = url.strip()  # Remove extra spaces
                if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                    url = urljoin("https://", url) # sets the urls starting from https://.....
                
                # parse the url
                parsed_url = urlparse(url)

                # check if the parsed url is valid
                if not parsed_url.scheme or not parsed_url.netloc:
                    print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
                    # continue
                else:
                
                    # try downloading images
                    try:
                        # Download the image
                        response = requests.get(url, stream=True) # send the url for downloading
                        response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                        
                        # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                        # If no file name is found, assigns a default name based on the index.
                        file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                        # Extracts the file extension (e.g., jpg). from the last index value
                        file_extension = file_name.split(".")[-1]
                        
                        # Ensure the file has a valid extension
                        if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                            file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                        
                        # Construct the full path for the file
                        full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}.jpg")
                        
                        # Save the image
                        with open(full_file_name, 'wb') as f:
                            for chunk in response.iter_content(1024):
                                f.write(chunk)

                        # Add watermark
                        add_watermark_to_image(full_file_name)
                        
                        print(f"Downloaded: {full_file_name}")
                    except Exception as e:
                        print(f"Failed to download {url} for {reg_no}: {e}")

# reports
def download_reports(data, main_folder="Inspection Report"): # here the main folder is Images
    
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # loop around every row to get the info
    for index, row in data.iterrows():
        reg_no = row["Reg"] # separate reg nums
        report_urls = row["Inspection Report"]
        
        # Create a subfolder for the car registration number
        # reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
        # os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
        # Check if the inspection report is missing
        if not report_urls or pd.isna(report_urls):
            print(f"Missing Inspection Report of {reg_no}")
            # continue
            
        else:
            if not report_urls.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
                report_urls = urljoin("https://", report_urls) # sets the urls starting from https://.....
            
            # parse the url
            parsed_url = urlparse(report_urls)
            
            # try downloading images
            try:
                # Download the image
                response = requests.get(report_urls, stream=True) # send the url for downloading
                response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
                # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
                # If no file name is found, assigns a default name based on the index.
                file_name = os.path.basename(parsed_url.path) or f"inspec_repo.pdf"

                # Extracts the file extension (e.g., pdf). from the last index value
                file_extension = file_name.split(".")[-1]
                
                # Ensure the file has a valid extension
                if file_extension not in ["pdf", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"inspec_repo.pdf" # set the extension if the extension is not in the above list
                
                # Construct the full path for the file
                full_file_name = os.path.join(main_folder, f"{reg_no}.pdf")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {report_urls} for {reg_no}:")

def start_funcs():
    thread1 = threading.Thread(target=download_images , args=(reg_img,))
    thread2 = threading.Thread(target=download_cond , args=(cond_img,))
    thread3 = threading.Thread(target=download_reports , args=(report,))

    thread1.start()
    thread2.start()
    thread3.start()
    
    thread1.join()
    thread2.join()
    thread3.join()

if __name__ == "__main__":
    start_funcs()


Watermark added to Images\LS71MTF\LS71MTF_1.jpg
Downloaded: Images\LS71MTF\LS71MTF_1.jpg
Watermark added to Images\LS71MTF\LS71MTF_2.jpg
Downloaded: Images\LS71MTF\LS71MTF_2.jpg
Watermark added to Images\LS71MTF\LS71MTF_3.jpg
Downloaded: Images\LS71MTF\LS71MTF_3.jpg
Watermark added to Images\LS71MTF\LS71MTF_4.jpg
Downloaded: Images\LS71MTF\LS71MTF_4.jpg
Watermark added to Damaged_images\LS71MTF\LS71MTF_1.jpg
Downloaded: Damaged_images\LS71MTF\LS71MTF_1.jpg
Watermark added to Images\LS71MTF\LS71MTF_5.jpg
Downloaded: Images\LS71MTF\LS71MTF_5.jpg
Watermark added to Images\LS71MTF\LS71MTF_6.jpg
Downloaded: Images\LS71MTF\LS71MTF_6.jpg
Watermark added to Images\LS71MTF\LS71MTF_7.jpg
Downloaded: Images\LS71MTF\LS71MTF_7.jpg
Watermark added to Images\LS71MTF\LS71MTF_8.jpg
Downloaded: Images\LS71MTF\LS71MTF_8.jpg
Downloaded: Inspection Report\LS71MTF.pdf
Watermark added to Images\LS71MTF\LS71MTF_9.jpg
Downloaded: Images\LS71MTF\LS71MTF_9.jpg
Watermark added to Images\LS71MTF\LS71MTF_10.jpg
Dow